# STEP file export, levelo of details

In [ ]:
import math
import xml.etree.ElementTree as ET

from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeTorus
from OCC.Extend.DataExchange.X3D import X3DLODShapeExporter
import OCC.Extend.DataExchange.x3d_standard.x3d as XX3D

from OCC.Extend.ShapeFactory import get_aligned_boundingbox
from OCC.Extend.DataExchange.STEP import read_step_file

from IPython.display import HTML

In [ ]:
# load STEP file
shp = read_step_file('./M8X30MM_ALLEN_KEY_SCREW.STEP')

In [ ]:
# set the level of details
# a dictionary with keys (distance from center, near to far), values (decimation ratio)
lod_lv = {2 * bbmax: 0., 3 * bbmax:0.4, 10 * bbmax:0.8}

In [ ]:
# lod exporter for the shape
lod_exp = X3DLODShapeExporter(shp, lod_levels=lod_lv)

In [ ]:
# X3D scene creation
x3dscene = XX3D.Scene(children=[])
x3ddoc = XX3D.X3D(Scene=x3dscene)
x3dscene.children.append(lod_exp._lod_Node)

In [ ]:
# render with x3dom
# zoom out to visualize the different meshes
# create x3dom html
_X3DOM_HEADER = '''<script type='text/javascript' src='https://www.x3dom.org/download/dev/x3dom-full.debug.js'> </script>
<link rel='stylesheet' type='text/css' href='https://www.x3dom.org/download/dev/x3dom.css'></link>
'''
x3d_element = list(ET.XML(x3ddoc.XML()).iter('X3D'))[0]

# remove gama correction
next(x3d_element.iter('Scene')).append(ET.XML('<Environment gammaCorrectionDefault="none"/>'))

# add crease angle to IndexedTriangleSet instances, supported by x3dom
# but not part of the X3D standard
for idx_ts in x3d_element.iter('IndexedTriangleSet'):
    idx_ts.set('creaseAngle', '0.2')

x3dHTML = ET.tostring(x3d_element, encoding="unicode", short_empty_elements=False)
x3dHTML = x3dHTML.replace("visible=", 'render=')
x3dom_html = _X3DOM_HEADER + x3dHTML

HTML(x3dom_html)